In [109]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np 
import os
import pandas as pd

In [110]:
os.getcwd()
os.chdir('C://Users//Jaesung//Desktop//해오라기//합칠거')

In [111]:
data = pd.read_csv('전처리완료.csv',engine ='python')

In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18398 entries, 0 to 18397
Data columns (total 11 columns):
안정성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_커피음료      18398 non-null int64
성장성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_치킨집       18398 non-null int64
서비스_업종_코드_명_패스트푸드점    18398 non-null int64
서비스_업종_코드_명_분식집       18398 non-null int64
cluster               18398 non-null int64
점포수                   18398 non-null int64
중년층_직장인구_수            18398 non-null int64
총_직장인구_수              18398 non-null int64
과밀_지수_구간_코드           18398 non-null float64
dtypes: float64(1), int64(10)
memory usage: 1.5 MB


In [113]:
X = data[data.columns[data.columns!='과밀_지수_구간_코드']]
Y = data[data.columns[data.columns=='과밀_지수_구간_코드']]

In [114]:
Y = pd.get_dummies(Y, columns=['과밀_지수_구간_코드'])

In [115]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

## modeling - NN

In [116]:
import tensorflow as tf
import math

In [117]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(12878, 10) (12878, 3)
(5520, 10) (5520, 3)


In [118]:
y_train.head()

,과밀_지수_구간_코드_0.0,과밀_지수_구간_코드_1.0,과밀_지수_구간_코드_2.0
5042,1,0,0
17279,0,0,1
2943,0,1,0
13544,0,0,1
13047,0,1,0


In [119]:
np.array(y_train)

array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [132]:
tf.reset_default_graph()

## Layer / Lr 는 데이터에 따라서 달라집니다.

In [133]:
#learning_rate = 0.0005  # 0.005
#learning_rate = 0.01
learning_rate = 0.001

In [134]:
X = tf.placeholder(tf.float32, shape=[None,10])
Y = tf.placeholder(tf.float32, shape=[None,3])

keep_prob = tf.placeholder(tf.float32)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)

dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)

dense3 = tf.layers.dense(inputs=dropout2, units=256, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)

logits= tf.layers.dense(inputs=dropout3, units=3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [135]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [136]:
model_path = 'C:/Users/Jaesung/Desktop/해오라기/my_test_model'
#saver = tf.train.Saver()
 
saver = tf.train.Saver()

### 미니배치 사이즈, 에폭은 데이터마다 달라집니다

In [137]:
batch_size = 64
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(1,100):
    avg_cost = 0
    concat = np.concatenate((X_train, y_train), axis=-1)
    np.random.shuffle(concat) #배치 셔플 진행 
    X_train = concat[:, :10].astype(np.float32)
    y_train = concat[:, 10:].astype(np.float32)
    for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
        x_ = X_train[batch_size * i: batch_size * (i + 1)]
        y_ = y_train[batch_size * i: batch_size * (i + 1)]
        feed_dict={X:x_, Y:y_, keep_prob:0.8}   
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c
    val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_train , keep_prob:1})  
    val_true = sess.run(tf.argmax(y_train,1))
    accuracy_val = accuracy_score(val_pred, val_true)
    if epoch% 20 == 0:
        #print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)))
        print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t train acc: {:.3f}%".format(accuracy_val * 100),)        
saver.save(sess, model_path)        

#if epoch% 20 == 0:
#   save

Epoch: 0020 cost =  0.014436481 	 train acc: 54.473%
Epoch: 0040 cost =  0.013410862 	 train acc: 60.304%
Epoch: 0060 cost =  0.013239136 	 train acc: 60.203%
Epoch: 0080 cost =  0.012929787 	 train acc: 61.228%


'C:/Users/Jaesung/Desktop/해오라기/my_test_model'

In [138]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) # keep_prob 삭제  
y_true = sess.run(tf.argmax(y_test,1))

In [139]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.629
Accuracy:  0.6289855072463768
             precision    recall  f1-score   support

          0       0.75      0.72      0.74      1883
          1       0.55      0.53      0.54      1751
          2       0.58      0.63      0.61      1886

avg / total       0.63      0.63      0.63      5520

[[1362  219  302]
 [ 282  925  544]
 [ 162  539 1185]]
